In [1]:
import pandas as pd
import numpy as np
import datetime as dt


In [2]:
faults_diagnostics = pd.read_csv('../data/faults_diagnostics.csv',
                                 index_col='RecordID',
                                 parse_dates = ['EventTimeStamp', 'LocationTimeStamp'],
                                 low_memory = False)

In [3]:
faults_diagnostics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 921737 entries, 1211418 to 1161752
Data columns (total 43 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   EventTimeStamp             921737 non-null  datetime64[ns]
 1   ESS_Id                     921737 non-null  int64         
 2   eventDescription           889644 non-null  object        
 3   actionDescription          0 non-null       float64       
 4   ecuSoftwareVersion         666527 non-null  object        
 5   ecuSerialNumber            623200 non-null  object        
 6   ecuModel                   867546 non-null  object        
 7   ecuMake                    867546 non-null  object        
 8   ecuSource                  921737 non-null  int64         
 9   spn                        921737 non-null  int64         
 10  fmi                        921737 non-null  int64         
 11  active                     921737 non-null  b

In [4]:
#prepare derates for dummization
faults_diagnostics.loc[faults_diagnostics['spn'] != 1569, 'spn_derate'] = 'neither' 
faults_diagnostics.loc[faults_diagnostics['spn'] != 5246, 'spn_derate'] = 'neither'
faults_diagnostics.loc[faults_diagnostics['spn'] == 5246, 'spn_derate'] = 'full'  
faults_diagnostics.loc[faults_diagnostics['spn'] == 1569, 'spn_derate'] = 'partial'  

#get dummies
derates = pd.get_dummies(faults_diagnostics['spn_derate'])

#merge back with original dataset
faults_diagnostics = faults_diagnostics.merge(derates, left_on = 'RecordID', right_on = 'RecordID').set_index('ESS_Id')



In [5]:
faults_diagnostics

,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,...,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_derate,full,neither,partial
ESS_Id,,,,,,,,,,,,,,,,,,,,,
108604426,2000-03-18 19:14:10,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,...,False,NaN,0.00000,NaN,100.0,0.58,neither,0,1,0
108604425,2000-03-18 19:14:10,High Voltage (Left Fuel Level Sensor),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,...,False,NaN,0.00000,NaN,100.0,0.58,neither,0,1,0
108604487,2000-03-18 19:20:47,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,0,1,0
108604488,2000-03-18 19:20:47,High Voltage (Left Fuel Level Sensor),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,829,3,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,0,1,0
108608408,2000-03-19 02:59:58,Not Reporting Data Wheel Sensor ABS Axle 2 Right,NaN,AAAI000032*AAAM000038*BB41275 *A82J140721A_9...,5W26153559,EC80ESP,BNDWS,11,792,7,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123901925,2020-03-06 13:44:45,Low (Severity Medium) Engine Coolant Level,NaN,04358814*06030918*051718174436*09401683*G1*BDR*,79904453,6X1u13D1500000000,CMMNS,0,111,18,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,0,1,0
123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,...,False,NaN,65.01096,NaN,73.2,7.83,partial,0,0,1
87903706,2026-05-16 14:44:11,NaN,NaN,unknown,unknown,unknown,unknown,0,5742,4,...,NaN,NaN,NaN,NaN,NaN,NaN,neither,0,1,0


In [6]:
faults_diagnostics['spn_derate'].unique()

array(['neither', 'partial', 'full'], dtype=object)

In [7]:
#extracting year out of datetimestamp
faults_diagnostics['EventTimeStamp'].dt.to_period('Y')

ESS_Id
108604426    2000
108604425    2000
108604487    2000
108604488    2000
108608408    2000
             ... 
123901925    2020
123905139    2020
87903706     2026
87903707     2026
87903705     2026
Name: EventTimeStamp, Length: 921737, dtype: period[A-DEC]

In [8]:
#extracting month/day
faults_diagnostics['EventTimeStamp'].dt.to_period('M')#('D')

ESS_Id
108604426    2000-03
108604425    2000-03
108604487    2000-03
108604488    2000-03
108608408    2000-03
              ...   
123901925    2020-03
123905139    2020-03
87903706     2026-05
87903707     2026-05
87903705     2026-05
Name: EventTimeStamp, Length: 921737, dtype: period[M]

In [9]:
#distribution per year
faults_diagnostics['EventTimeStamp'].dt.to_period('Y').value_counts()

2016    283209
2015    239825
2017    210993
2018    108005
2019     69563
2020      9777
2011       176
2000       129
2010        37
2009        19
2026         3
2002         1
Freq: A-DEC, Name: EventTimeStamp, dtype: int64

In [10]:
faults_diagnostics=faults_diagnostics.drop(['actionDescription', 'ServiceDistance', 'faultValue'], axis=1).reset_index(drop=True)


In [11]:
faults_diagnostics=faults_diagnostics.fillna(0)

#### 1,064 with full derates (spn=5246)-- (201 trucks)
#### 10,711 partial derates (spn=1569)-- (498 trucks)
#### 921,737 no derates (spn != 5246/1569)-- (517 trucks)

#### 182 trucks have both partial and full derates


In [12]:
#we have 1064 full derates
full_derates=faults_diagnostics[faults_diagnostics['spn']==5246]
full_derates

,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,...,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_derate,full,neither,partial
217,2011-01-01 00:03:22,0,04384413*22115617*090617144354*60701715*G1*BGT*,80035307,6X1u17D1500000000,CMMNS,0,5246,16,True,...,6143.0,True,0.000000,0.0,100.0,0.00,full,1,0,0
223,2011-01-01 00:03:23,0,04384413*22115617*090617144354*60701715*G1*BGT*,80035307,6X1u17D1500000000,CMMNS,0,5246,0,True,...,6143.0,True,0.000000,0.0,100.0,0.00,full,1,0,0
230,2011-01-01 00:03:34,0,05317106*04101156*020516150530*09400053*G1*BDR*,79743733,6X1u13D1500000000,CMMNS,0,5246,0,True,...,22527.0,False,9.689507,0.0,100.0,1.45,full,1,0,0
238,2011-01-01 00:04:57,0,04993120*00041643*042114185815*07700062*I0*BBZ*,79475366,6X1u10D1500000000,CMMNS,0,5246,0,True,...,22527.0,True,0.000000,0.0,0.0,0.00,full,1,0,0
257,2011-01-01 00:16:06,0,04993120*00041643*042114185815*07700062*I0*BBZ*,79475366,6X1u10D1500000000,CMMNS,0,5246,0,True,...,22527.0,0,0.000000,0.0,0.0,0.00,full,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919342,2020-02-19 07:02:33,0,05317106*05005224*051718172255*09401583*G1*BDR*,79845785,6X1u13D1500000000,CMMNS,0,5246,0,True,...,22527.0,0,0.000000,0.0,100.0,0.58,full,1,0,0
919678,2020-02-21 07:23:44,0,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,True,...,18431.0,True,0.000000,0.0,100.0,0.00,full,1,0,0
920081,2020-02-24 15:27:26,0,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,0,True,...,5119.0,True,0.000000,0.0,100.0,0.00,full,1,0,0
920083,2020-02-24 15:28:05,0,04384413*22246857*090619141107*60701756*G1*BGT*,80092582,6X1u17D1500000000,CMMNS,0,5246,16,False,...,5119.0,0,0.000000,0.0,0.0,0.00,full,1,0,0


In [13]:
full_derates['EquipmentID'].nunique()

201

In [14]:
full_derates['EquipmentID'].unique()

array([2175, 1751, 1452, 1395, 1630, 1487, 1329, 1419, 1486, 1509, 1492,
       1473, 1383, 1396, 1602, 1431, 1795, 1595, 1366, 1339, 1391, 1399,
       1417, 1591, 1490, 1547, 1444, 1457, 1696, 1437, 1453, 1584, 1692,
       1590, 1398, 1418, 1472, 1560, 1572, 1375, 1742, 1566, 1778, 1524,
       1659, 1403, 1535, 1567, 1373, 1665, 1551, 1440, 1571, 1443, 1378,
       1488, 1384, 1582, 1508, 1599, 1501, 1407, 1603, 1604, 1477, 1389,
       1519, 1757, 1458, 1401, 1467, 1579, 1534, 1575, 1539, 1601, 1657,
       1592, 1563, 1557, 1605, 1549, 1581, 1811, 1573, 1594, 1668, 1643,
       1561, 1940, 1839, 1552, 1637, 1554, 1686, 1598, 1828, 1525, 1848,
       1521, 1661, 1831, 1528, 2011, 1654, 1917, 1543, 1812, 1803, 1814,
       1556, 1564, 1585, 1663, 1623, 1800, 1823, 1732, 1818, 1586, 2009,
       1946, 1559, 1711, 1600, 1939, 1704, 1808, 1743, 1822, 1739, 1764,
       1846, 2007, 1669, 1819, 1780, 2143, 1718, 2111, 1874, 1689, 1932,
       1731, 1794, 1978, 1691, 1942, 1621, 1928, 18

In [15]:
#we have 10711 partial derates
partial_derates=faults_diagnostics[faults_diagnostics['spn']==1569]
partial_derates

,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,...,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_derate,full,neither,partial
28,2000-03-19 14:02:34,Condition Exists Engine Protection Torque Derate,04358814*06086133*051718174436*09401683*G1*BDR*,79923861,6X1u13D1500000000,CMMNS,0,1569,31,True,...,18431.0,False,62.438100,0.00,100.0,25.81,partial,0,0,1
229,2011-01-01 00:03:34,Condition Exists Engine Protection Torque Derate,05317106*04101156*020516150530*09400053*G1*BDR*,79743733,6X1u13D1500000000,CMMNS,0,1569,31,True,...,22527.0,False,9.689507,0.00,100.0,1.45,partial,0,0,1
246,2011-01-01 00:05:58,Condition Exists Engine Protection Torque Derate,unknown,unknown,unknown,unknown,0,1569,31,False,...,1023.0,0,0.000000,0.00,0.0,0.00,partial,0,0,1
397,2015-02-21 12:06:22,Condition Exists Engine Protection Torque Derate,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,...,18431.0,False,51.971870,3276.75,0.0,0.58,partial,0,0,1
547,2015-02-21 15:01:49,Condition Exists Engine Protection Torque Derate,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,False,...,17407.0,0,0.000000,0.00,0.0,0.00,partial,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921005,2020-03-02 15:51:18,Condition Exists Engine Protection Torque Derate,04358814*06009282*100515184137*09400147*G1*BDR*,79897319,6X1u13D1500000000,CMMNS,0,1569,31,True,...,18431.0,False,1.640808,0.00,100.0,0.58,partial,0,0,1
921082,2020-03-03 06:51:50,Condition Exists Engine Protection Torque Derate,05317106*05088449*051718172255*09401583*G1*BDR*,79875897,6X1u13D1500000000,CMMNS,0,1569,31,False,...,22527.0,0,0.000000,0.00,0.0,0.00,partial,0,0,1
921092,2020-03-03 06:59:13,Condition Exists Engine Protection Torque Derate,05317106*05088449*051718172255*09401583*G1*BDR*,79875897,6X1u13D1500000000,CMMNS,0,1569,31,False,...,22527.0,0,0.000000,0.00,0.0,0.00,partial,0,0,1
921346,2020-03-04 16:55:48,Condition Exists Engine Protection Torque Derate,04358814*06027162*051718174436*09401683*G1*BDR*,79902732,6X1u13D1500000000,CMMNS,0,1569,31,False,...,1023.0,0,0.000000,0.00,0.0,0.00,partial,0,0,1


In [16]:
partial_derates['EquipmentID'].nunique()

498

In [17]:
partial_derates['EquipmentID'].unique()

array([1968, 1751, 1917, 1721, 1515, 1630, 1395, 1487, 1499, 1329, 1365,
       1419, 1486, 1509, 1492, 1340, 1328, 1430, 1370, 1497, 1350, 1608,
       1606, 1612, 1354, 1493, 1583, 1465, 1335, 1789, 1383, 1389, 1501,
       1438, 1463, 1514, 1666, 1689, 1692, 1357, 1396,  310, 1602, 1660,
       1445, 1431, 1327, 1795, 1399, 1494, 1367, 1364, 1452, 1595, 1740,
       1366, 1339, 1425, 1358, 1646, 1391, 1428, 1338, 1637, 1346, 1490,
       1562, 1433, 1663, 1417, 1424, 1771, 1790, 1591, 1797, 1596, 1421,
       1444, 1480, 1466, 1457, 1794, 1696, 1703, 1626, 1715, 1437, 1453,
       1584, 1597, 1610, 1377, 1652, 1517, 1784, 1786, 1590, 1341, 1382,
       1714, 1711, 1369, 1398, 1791, 1772, 1472, 1473, 1418, 1645, 1475,
       1815, 1806, 1607, 1560, 1450, 1401, 1572, 1375, 1368, 1650, 1443,
       1495, 1670, 1587, 1778, 1566, 1617, 1691, 1573, 1477, 1476, 1441,
       1405, 1577, 1447, 1659, 1408, 1439, 1403, 1458, 1668, 1814, 1567,
       1373, 1446, 1700, 1629, 1730, 1665, 1381, 15

In [18]:
#no derates= 921737 (spn!-1569, 5246)
neither=faults_diagnostics[(faults_diagnostics['spn']!=1569) | (faults_diagnostics['spn']!=5246)]
neither

,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,...,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_derate,full,neither,partial
0,2000-03-18 19:14:10,High Voltage (Fuel Level),0,0,CECU3B-NAMUX4,PACCR,49,96,3,True,...,1279.0,False,0.00000,0.0,100.0,0.58,neither,0,1,0
1,2000-03-18 19:14:10,High Voltage (Left Fuel Level Sensor),0,0,CECU3B-NAMUX4,PACCR,49,829,3,True,...,1279.0,False,0.00000,0.0,100.0,0.58,neither,0,1,0
2,2000-03-18 19:20:47,High Voltage (Fuel Level),0,0,CECU3B-NAMUX4,PACCR,49,96,3,False,...,255.0,0,0.00000,0.0,0.0,0.00,neither,0,1,0
3,2000-03-18 19:20:47,High Voltage (Left Fuel Level Sensor),0,0,CECU3B-NAMUX4,PACCR,49,829,3,False,...,255.0,0,0.00000,0.0,0.0,0.00,neither,0,1,0
4,2000-03-19 02:59:58,Not Reporting Data Wheel Sensor ABS Axle 2 Right,AAAI000032*AAAM000038*BB41275 *A82J140721A_9...,5W26153559,EC80ESP,BNDWS,11,792,7,False,...,1279.0,0,0.00000,0.0,0.0,0.00,neither,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921732,2020-03-06 13:44:45,Low (Severity Medium) Engine Coolant Level,04358814*06030918*051718174436*09401683*G1*BDR*,79904453,6X1u13D1500000000,CMMNS,0,111,18,False,...,1023.0,0,0.00000,0.0,0.0,0.00,neither,0,1,0
921733,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,...,18431.0,False,65.01096,0.0,73.2,7.83,partial,0,0,1
921734,2026-05-16 14:44:11,0,unknown,unknown,unknown,unknown,0,5742,4,False,...,1023.0,0,0.00000,0.0,0.0,0.00,neither,0,1,0
921735,2026-05-16 14:44:11,0,unknown,unknown,unknown,unknown,0,5743,4,False,...,1023.0,0,0.00000,0.0,0.0,0.00,neither,0,1,0


In [19]:
#517 trucks don't have derates
neither['EquipmentID'].nunique()

517

In [20]:
#182 trucks have both full and partial derates 
both_derates= pd.Series(list(set(full_derates['EquipmentID']).intersection(set(partial_derates['EquipmentID']))))
both_derates

0      1549
1      1551
2      1552
3      1554
4      1556
       ... 
177    1501
178    1508
179    1509
180    2021
181    1519
Length: 182, dtype: int64

In [21]:
both_derates.unique()

array([1549, 1551, 1552, 1554, 1556, 1557, 1559, 1560, 1561, 1563, 1564,
       1566, 1567, 1571, 1572, 1573, 1575, 1579, 1581, 1582, 1584, 1585,
       1586, 1590, 1591, 1592, 1594, 1595, 1598, 1599, 1600, 1601, 1602,
       1603, 1604, 1605, 1621, 1623, 1630, 1637, 1643, 1654, 1657, 1659,
       1661, 1663, 1665, 1668, 1669, 1683, 1686, 1689, 1691, 1692, 1696,
       1698, 1704, 1711, 1718, 1731, 1732, 1739, 1743, 1751, 1757, 1758,
       1764, 1768, 1772, 1778, 1780, 1794, 1795, 1797, 1800, 1803, 1808,
       1811, 1812, 1814, 1815, 1818, 1819, 1822, 1823, 1827, 1828, 1831,
       1834, 1839, 1329,  306, 1842, 1846, 1847, 1848, 1339, 1853, 1854,
       1856, 1857, 1858, 1860, 1864, 1871, 1872, 1874, 1877, 1366, 1878,
       1879, 1373, 1885, 1375, 1378, 1383, 1384, 1896, 1389, 1391, 1395,
       1396, 1907, 1398, 1399, 1401, 1403, 1917, 1918, 1407, 1919, 1921,
       1922, 1928, 1417, 1418, 1419, 1932, 1939, 1940, 1942, 1431, 1945,
       1946, 1947, 1437, 1950, 1951, 1440, 1443, 14

In [26]:
pd.set_option('display.max_rows', None)

In [27]:
faults_diagnostics[faults_diagnostics['EquipmentID']==1396]

,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,...,LampStatus,ParkingBrake,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_derate,full,neither,partial
2951,2015-02-24 06:32:33,Low (Severity Low) Engine Coolant Level,04993120*00027785*040213150018*07700044*I0*BBZ*,79464671,6X1u10D1500000000,CMMNS,0,111,17,True,...,1023.0,0,0.000000,3276.75,0.0,1.16,neither,0,1,0
2963,2015-02-24 06:38:38,Low (Severity Low) Engine Coolant Level,04993120*00027785*040213150018*07700044*I0*BBZ*,79464671,6X1u10D1500000000,CMMNS,0,111,17,False,...,1023.0,0,0.000000,0.00,0.0,0.00,neither,0,1,0
12659,2015-04-12 08:48:39,Low (Severity Low) Engine Coolant Level,04993120*00027785*040213150018*07700044*I0*BBZ*,79464671,6X1u10D1500000000,CMMNS,0,111,17,True,...,1023.0,0,0.000000,0.00,0.0,0.00,neither,0,1,0
12915,2015-04-12 17:50:53,Low (Severity Low) Engine Coolant Level,04993120*00027785*040213150018*07700044*I0*BBZ*,79464671,6X1u10D1500000000,CMMNS,0,111,17,False,...,1023.0,0,0.000000,0.00,0.0,0.00,neither,0,1,0
12962,2015-04-12 20:30:08,Low (Severity Low) Engine Coolant Level,04993120*00027785*040213150018*07700044*I0*BBZ*,79464671,6X1u10D1500000000,CMMNS,0,111,17,False,...,1023.0,0,0.000000,0.00,0.0,0.00,neither,0,1,0
13909,2015-04-14 04:57:23,Low (Severity Low) Engine Coolant Level,04993120*00027785*040213150018*07700044*I0*BBZ*,79464671,6X1u10D1500000000,CMMNS,0,111,17,True,...,1023.0,0,0.000000,0.00,0.0,0.00,neither,0,1,0
13910,2015-04-14 05:00:58,Low (Severity Low) Engine Coolant Level,04993120*00027785*040213150018*07700044*I0*BBZ*,79464671,6X1u10D1500000000,CMMNS,0,111,17,False,...,1023.0,0,0.000000,0.00,0.0,0.00,neither,0,1,0
16494,2015-04-17 01:54:54,Low (Severity Low) Engine Coolant Level,04993120*00027785*040213150018*07700044*I0*BBZ*,79464671,6X1u10D1500000000,CMMNS,0,111,17,True,...,1023.0,0,0.000000,3276.75,0.0,3.48,neither,0,1,0
16495,2015-04-17 02:06:29,Low (Severity Low) Engine Coolant Level,04993120*00027785*040213150018*07700044*I0*BBZ*,79464671,6X1u10D1500000000,CMMNS,0,111,17,False,...,1023.0,0,0.000000,0.00,0.0,0.00,neither,0,1,0
17421,2015-04-18 05:48:29,Low (Severity Low) Engine Coolant Level,04993120*00027785*040213150018*07700044*I0*BBZ*,79464671,6X1u10D1500000000,CMMNS,0,111,17,True,...,1023.0,0,0.000000,0.00,0.0,0.00,neither,0,1,0
